# 1 - DATA PROCESS
------


In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib import colors as mcolors
import numpy as np
import os
import glob
from io import StringIO
from scipy.stats import linregress
import matplotlib.cm as cm
import shutil
import json
import itertools

---------
## Función `normalize(file_paths, capitalize_c=False)`

**Finalidad:**  
Convierte los archivos de texto a un formato uniforme, reemplazando comas por puntos decimales. Además, puede renombrar archivos cuyo nombre empiece por "c" para que empiecen por "C".  

**Variables utilizadas:**  
- `file_paths`: lista de rutas de archivos a procesar.  
- `capitalize_c`: booleano que controla si se renombran los archivos que empiezan con "c".  
- Variables internas como `dirname`, `basename`, `new_basename`, `new_path` se usan para manejar rutas y renombrar.  

**Inputs:**  
- `file_paths`: lista de strings con rutas de archivos `.txt`.  
- `capitalize_c`: opcional (por defecto `False`), si se activa renombra los archivos.  

**Outputs:**  
- No devuelve nada directamente.  
- Modifica los archivos en disco (reemplaza comas por puntos y, opcionalmente, renombra).  
- Muestra mensajes en consola con el progreso o errores.  


In [ ]:
def normalize(file_paths, capitalize_c = False):
    for path in file_paths:
        try:
            with open(path, 'r', encoding='utf-8') as f: #cambia de comas a puntos
                content = f.read()
            updated = content.replace(',', '.')

            with open(path, 'w', encoding='utf-8') as f:
                f.write(updated)
            
            
            #print(f"Updated in place: {path}")
            
            if capitalize_c:
                dirname = os.path.dirname(path)
                basename = os.path.basename(path)
                
                if basename.startswith('c'):
                    new_basename = 'C' + basename[1:]
                    new_path = os.path.join(dirname, new_basename)
                    
                    os.rename(path, new_path)
                    print(f"Renamed file: {path} → {new_path}")
                    path = new_path  # Update path for future processing

        except Exception as e:
            print(f"Error processing {path}: {e}")


---------------
## Función `thresholds(filepath)`

**Finalidad:**  
Lee de un archivo de texto los valores de Upper y Lower Threshold.  

**Variables utilizadas:**  
- `filepath`: ruta del archivo.  
- `upper`, `lower`: se inicializan en `None` y se actualizan al leer las líneas correspondientes.  

**Inputs:**  
- `filepath`: string con la ruta del archivo `.txt`.  

**Outputs:**  
- Una tupla `(upper, lower)` con los umbrales en formato `float`.  


In [ ]:
def thresholds(filepath):
    upper, lower = None, None
    with open(filepath, 'r') as f:
        for line in f:
            if 'Upper Threshold:' in line:
                upper = float(line.split('\t')[1].strip())
            elif 'Lower Threshold:' in line:
                lower = float(line.split('\t')[1].strip())
    return upper, lower

---------------
## Función `clean_data(filepath, delimiter='\t')`

**Finalidad:**  
Carga los datos de un archivo de texto, limpiando líneas vacías y descartando las no numéricas. Devuelve un DataFrame con las columnas de tiempo y amplitud.  

**Variables utilizadas:**  
- `filepath`: ruta al archivo.  
- `delimiter`: separador usado en el archivo (por defecto tabulador).  
- `rows`: lista acumuladora de los pares [tiempo, amplitud].  

**Inputs:**  
- `filepath`: ruta del archivo de datos.  
- `delimiter`: opcional, separador de columnas (default: tabulador `\t`).  

**Outputs:**  
- DataFrame de pandas con columnas `['Time', 'Amplitude']`.  


In [ ]:
def clean_data(filepath, delimiter='\t'):
    rows = []

    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue

            parts = line.split(delimiter)
            while len(parts) < 2:
                parts.append('')

            try:
                time_val = float(parts[0].strip())
                amplitude_val = float(parts[1].strip())
                rows.append([time_val, amplitude_val])
            except ValueError:
                continue  # Skip non-numeric lines

    return pd.DataFrame(rows, columns=['Time', 'Amplitude'])


---------------
## Función `extract_amplitude(df)`

**Finalidad:**  
Extrae un segmento del DataFrame correspondiente a un ciclo de amplitud, delimitado entre la segunda y tercera aparición del valor de tiempo 0.000. Este segmento corresponde al eje Anterior-Posterior de la respiración del paciente practicando el DIBH.  

**Variables utilizadas:**  
- `df`: DataFrame de entrada.  
- `zero_indices`: posiciones donde `Time ≈ 0`.  
- `start`, `end`: índices que marcan los límites del segmento.  

**Inputs:**  
- `df`: DataFrame con columnas `['Time','Amplitude']`.  

**Outputs:**  
- Sub-DataFrame con el rango de interés, reindexado.  


In [ ]:
def extract_amplitude(df):
    # Find indices where Time is exactly (or very close to) 0.000
    zero_indices = df.index[abs(df['Time']) < 1e-6].tolist()
    
    # Ensure there are at least 3 zero crossings
    if len(zero_indices) < 3:
        raise ValueError("Dataframe doesn't contain at least three 0.000 entries in 'Time' column")
    
    start = zero_indices[1]
    end = zero_indices[2]
    
    # Extract and return the segment
    return df.iloc[start:end].reset_index(drop=True)

---------------
## Función `extract_activation_data(filepaths=None)`

**Finalidad:**  
Extrae los eventos de activación del haz ("Beam Enable/Disable Moments") de uno o varios archivos y los devuelve en un DataFrame.  

**Variables utilizadas:**  
- `filepaths`: lista de rutas a archivos.  
- `activation_rows`: lista con pares [tiempo, valor de beam, archivo origen].  
- `in_activation_section`: bandera para saber si se está leyendo la sección correcta.  

**Inputs:**  
- `filepaths`: lista de archivos, string con un archivo, o `None` (si es None busca `Camp*.txt`).  

**Outputs:**  
- DataFrame con columnas `['Time','Beam','Source_File']`.  


In [ ]:
def extract_activation_data(filepaths=None):
    if filepaths is None:
        filepaths = sorted(glob.glob('Camp*.txt'))
    elif isinstance(filepaths, str):
        filepaths = [filepaths]
    if len(filepaths) < 1:
        raise ValueError("Need at least 1 file to extract activation data")

    activation_rows = []
    for filepath in filepaths:
        in_activation_section = False
        with open(filepath, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if "Beam Enable/Disable Moments" in line:
                    in_activation_section = True
                    continue
                if in_activation_section and line:
                    parts = line.split('\t')
                    if len(parts) >= 2:
                        try:
                            time_val = float(parts[0].strip())
                            activation_val = int(float(parts[1].strip()))  # robust: acepta 1 o 1.0
                            activation_rows.append([time_val, activation_val, filepath])
                        except ValueError:
                            continue
    return pd.DataFrame(activation_rows, columns=['Time', 'Beam', 'Source_File'])


---------------
## Función `check_overlap(df1, df2, tol=0.005, window=10, min_matches=300)`

**Finalidad:**  
Detecta si hay solapamiento entre el final de `df1` y el inicio de `df2`, comparando amplitudes en una ventana de tiempo.

La lógica de la función es, ya que sabemos que siempre que se solapen, lo hacen alrededor de 8s del final de la `df1` con 8s del principio de la `df2`, usamos la amplitud del `idx=0` de la `df2` para ir mirando cada valor desde los ultimos `window` segundos para ver si hay algun punto en el que las amplitudes coincidan con una diferencia máxima de `tol`. Si coinciden en el indice `idx'=i` de la df1, se hace `+=1` a `matches` y se compara el `idx=1` de la `df2` con el `idx'=i+1` de la `df1` y asi sucesivamente hasta el ultimo indice posible de `df1`. Si al finalizar hay mas de `min_matches` coincidencias, se determina que si que hay overlap, sinó, suponemos que las coincidencias han sido aleatorias y no hay overlap.

**Variables utilizadas:**  
- `df1`, `df2`: DataFrames con datos de amplitud extraidos con las funciones clean_data i extract_amplitude.
- `tol`: tolerancia en amplitud (cm) a partir de la cual se determina un match.
- `window`: tamaño de la ventana (s) en la que puede aparecer el overlap.
- `min_matches`: número mínimo de coincidencias para determinat si hay overlap.
- Variables intermedias: `t1`, `a1`, `t2`, `a2`, `a1_window`.  

**Inputs:**  
- `df1`, `df2`: DataFrames con columnas `['Time','Amplitude']`.  
- `tol`: tolerancia de coincidencia (default 0.005).  
- `window`: segundos a revisar (default 10).  
- `min_matches`: mínimo de coincidencias (default 300).  

**Outputs:**  
- Tupla `(True, idx)` si hay solapamiento, indicando el índice de inicio de solapamiento en `df1`.  
- Tupla `(False, None)` si no se encuentra solapamiento.  


In [ ]:
def check_overlap(df1, df2, tol=0.005, window=10, min_matches=300):

    #dataframes con los valores de ambos df separados
    t1 = df1['Time'].values
    a1 = df1['Amplitude'].values
    t2 = df2['Time'].values
    a2 = df2['Amplitude'].values

    t_max = t1[-1] #coge el último indice del df1: el ultimo dato tomado en el df1
    mask = t1 >= t_max - window #nuevo df en el que solo van a estar los indices que corresponden a los ultimos 'window' segundos
    a1_window = a1[mask] #acortamos a_1 a solo los valores de los ultimos 10 segundos de df1

    # buscar coincidencia entre final de df1 y comienzo de df2
    for start in range(len(a1_window)):
        length = min(len(a1_window) - start, len(a2))
        diffs = np.abs(a1_window[start:start+length] - a2[:length])
        matches = np.sum(diffs <= tol)
        if matches >= min_matches: #si hay mas de 300 matches, se considera que hay solapamiento
            idx_start = np.where(mask)[0][start]
            return True, idx_start
    return False, None

---------------
## Función `build_df_with_beam(filepath)`

**Finalidad:**  
El objetivo de esta función es juntar en un único Dataframe los dataframes de amplitud i del beam.

La lógica de la función es primeramente crear los dataframes con las funciones `extract_amplitude` y `extract_activation_data`.
Juntamos en una Serie todos los tiempos (los de ambos dataframes) para añadirla a un Dataframe de 3 columnas: `['Time','Amplitude','Beam']`. 

El `df_amp` tiene muestras en tiempos cada 0.015s, mientras que `df_beam` tiene unicamente muestras cuando detecta que puede haber activación o desactivación. Por tanto, las muestras es practicamente imposible que coincidan en tiempo. Esto produce que, cuando se añadan las columnas del `df_amp` i del `df_beam` al `df_total`, haya un `NaN` en la columna de las amplitudes para los tiempos de activación y miles de `NaN` en la columna de beam para los tiempos que no son de activación. \
Para solucionarlo, simplemente hacemos un `fill forward` con una función de pandas para que, si detecta un `NaN`, copie el valor anterior no nulo en ese indice. Si hacemos este proceso para ambas columnas, conseguimos rellenar todo el dataframe de valores. 

Hay un caso particular que hay que tener en cuenta a veces. No siempre hay un valor de activación para `t=0`, por lo que se definen como `NaN` todos los valores de activación hasta que hay una. Para solucionarlo, se hace un `fill backwards` con el valor opuesto al primer `no-NaN` que se encuentre en la columna de activación: si el primer valor es un 1, rellena el dataframe hacia atras con 0, y viceversa.

**Variables utilizadas:**  
- `df_amp`: amplitud extraída con `extract_amplitude`.  
- `df_beam`: datos de beam extraídos con `extract_activation_data`.  
- `df_total`: DataFrame combinado y rellenado.  

**Inputs:**  
- `filepath`: ruta del archivo `.txt`.  

**Outputs:**  
- DataFrame conjunto con columnas `['Time','Amplitude','Beam']`.  


In [ ]:
def build_df_with_beam(filepath):

    # --- Amplitud ---
    df_amp = extract_amplitude(clean_data(filepath))

    # --- Eventos Beam crudos ---
    df_beam = extract_activation_data(filepath)[["Time", "Beam"]].sort_values("Time")
    
    # 1. Unir todos los tiempos y ordenarlos
    all_times = pd.Series(sorted(set(df_amp["Time"]).union(set(df_beam["Time"]))), name="Time")
    
    # 2. Crear dataframe base
    df_total = pd.DataFrame(all_times)
    df_total["Amplitude"] = pd.NA
    df_total["Beam"] = pd.NA

    # 3. Insertar valores conocidos de Amplitude
    df_total = df_total.merge(df_amp, on="Time", how="left", suffixes=("", "_amp"))
    df_total["Amplitude"] = df_total["Amplitude"].combine_first(df_total["Amplitude_amp"])
    df_total = df_total.drop(columns=["Amplitude_amp"])

    # 4. Insertar valores conocidos de Beam
    df_total = df_total.merge(df_beam, on="Time", how="left", suffixes=("", "_beam"))
    df_total["Beam"] = df_total["Beam"].combine_first(df_total["Beam_beam"])
    df_total = df_total.drop(columns=["Beam_beam"])

    # 5. Rellenar hacia adelante
    df_total = df_total.sort_values("Time").reset_index(drop=True)
    df_total["Amplitude"] = df_total["Amplitude"].ffill()
    df_total["Beam"] = df_total["Beam"].ffill()
    
    # 6. Manejo especial del primer valor de Beam si es NaN
    if pd.isna(df_total.loc[0, "Beam"]):
        first_valid = df_total["Beam"].first_valid_index()
        if first_valid is not None:
            first_val = df_total.loc[first_valid, "Beam"]
            if first_val == 1:
                # Si el primer valor válido es 1, rellenamos con 0 hacia atrás
                df_total.loc[:first_valid, "Beam"] = df_total["Beam"].bfill()
                df_total.loc[:first_valid-1, "Beam"] = 0
            elif first_val == 0:
                # Si el primer valor válido es 0, rellenamos con 1 hacia atrás
                df_total.loc[:first_valid, "Beam"] = df_total["Beam"].bfill()
                df_total.loc[:first_valid-1, "Beam"] = 1

    return df_total

---------------
## Función `merge_timelines(files, tol=0.005, window=10, min_matches=300)`

**Finalidad:**  
El objetivo de esta función es concatenar todos los archivos de una misma sessión en un unico archivo, segregando los caso de overlap y los de no y guardando los valores de threshold. Todo en un unico archivo.

La lógica de la función es crear un dataframe vació e ir añadiendo los dataframes de cada archivo a este conjunto.

Primero se añadiran en una fila de indice 0 los valores de upper i lower threshold para poder guardarlos.

A partir de ahi, la idea es añadir el primer dataframe y pasar la función `check_overlap`. Si hay overlap, se eliminan los datos del primer archivo desde el overlap hasta el final y se calcula el `timeshift` que se debe sumar al segundo archivo para que la linea temporal empalme perfectamente, asi sucesivamente hasta empalmarlos todos. Si noo hay overlap, como los archivos ya empalman de por si, simplemente es sumarle el tiempo del ultimo indice del primer dataframe al segundo, y asi crear una linea temporal continua.

**Variables utilizadas:**  
- `files`: lista de rutas a archivos.  
- `df_total`: acumulador de los DataFrames concatenados.  
- `upper, lower`: thresholds leídos del primer archivo.  
- `meta_row`: primera fila con los thresholds.  
- Variables intermedias: `overlap`, `idx`, `time_shift`, `df_new_shifted`.  

**Inputs:**  
- `files`: lista de rutas de archivos `.txt`.  
- `tol`, `window`, `min_matches`: parámetros de detección de solapamiento.  

**Outputs:**  
- DataFrame final con columnas `['Time','Amplitude','Beam','UpperThreshold','LowerThreshold']`, incluyendo la fila inicial con thresholds.  


In [ ]:
def merge_timelines(files, tol=0.005, window=10, min_matches=300):
    
    if not files:
        raise FileNotFoundError("No se encontraron archivos .txt en la carpeta")

    normalize(files)
    df_total = pd.DataFrame(columns=["Time", "Amplitude", "Beam","UpperThreshold","LowerThreshold"])
    upper, lower = thresholds(files[0])
    meta_row = pd.DataFrame([{
    "Time": None,
    "Amplitude": None,
    "Beam": None,
    "UpperThreshold": upper,
    "LowerThreshold": lower
    }])

    for i, f in enumerate(files):
        df_new = build_df_with_beam(f)

        if df_total.empty:
            df_total = df_new.copy()
            continue

        # comprobamos solapamiento
        overlap, idx = check_overlap(df_total, df_new, tol=tol, window=window, min_matches=min_matches)

        if overlap:
            # tiempo de inicio del solapamiento en df_total
            t_overlap = df_total.iloc[idx]["Time"]
            # inicio del nuevo archivo
            t_new_start = df_new["Time"].iloc[0]
            # calculamos shift para alinear inicio del nuevo con el inicio de solapamiento
            time_shift = t_overlap - t_new_start

            df_new_shifted = df_new.copy()
            df_new_shifted["Time"] = df_new["Time"] + time_shift

            #print(f"✅ Overlap detectado en {f} con inicio en t = {t_overlap:.3f} s (shift={time_shift:.6f})")

            # reemplazamos desde idx en adelante por el nuevo archivo
            df_total = pd.concat([df_total.iloc[:idx], df_new_shifted], ignore_index=True)

        else:
            # si no hay overlap, el archivo nuevo empieza después del último tiempo
            last_time = df_total["Time"].iloc[-1]
            dt = np.median(df_new["Time"].diff().dropna())
            if not np.isfinite(dt) or dt <= 0:
                dt = 1e-6
            time_shift = last_time - df_new["Time"].iloc[0] + dt

            df_new_shifted = df_new.copy()
            df_new_shifted["Time"] = df_new["Time"] + time_shift

            #print(f"❌ No overlap detectado en {f}. Shift aplicado = {time_shift:.6f}")

            df_total = pd.concat([df_total, df_new_shifted], ignore_index=True)

    # opcional: redondear tiempos para evitar problemas de floats (si quieres mantenerlo)
    df_total["Time"] = df_total["Time"].round(6)
    df_final = pd.concat([meta_row, df_total], ignore_index=True)

    return df_final

--------
## Función `cut_delay(df)`

**Finalidad:**  
Recorta el primer segundo de cada activación del `Beam`.  
En los datos de DIBH, cuando la columna `Beam` pasa de `0` a `1`, significa que el haz de radiación puede habilitarse, pero en la práctica no se activa hasta **1 segundo después**. Esta función corrige ese desfase: en cada transición 0→1, pone la columna `Beam=0` desde el instante de transición `t` hasta `t+1`.  

**Variables utilizadas:**  
- `df`: DataFrame extraido de la función `merge_timelines`.
- `times`: array con los valores de tiempo (`df['Time'].values`).  
- `beam`: array con los valores de la columna `Beam`.  
- `transitions`: array booleano que detecta las posiciones en que `Beam` pasa de 0 a 1.  
- `transition_indices`: índices de las transiciones 0→1.  
- `mask`: máscara booleana que selecciona las filas entre `t` y `t+1s` después de cada transición.  

**Inputs:**  
- `df`: DataFrame con las columnas `['Time','Amplitude','Beam','UpperThreshold','LowerThreshold']`.  

**Outputs:**  
- `df_mod`: DataFrame modificado, idéntico al original salvo que en la columna `Beam` se corrige el retraso de 1s después de cada transición 0→1. 

In [ ]:
def cut_delay(df: pd.DataFrame) -> pd.DataFrame:
    
    df_mod = df.copy()
    times = df_mod["Time"].values
    beam = df_mod["Beam"].values

    # Detectar transiciones 0 -> 1
    transitions = (beam[1:] == 1) & (beam[:-1] == 0)
    transition_indices = transitions.nonzero()[0] + 1  # desplazamos +1 porque es la segunda muestra la que pasa a 1

    for idx in transition_indices:
        t0 = times[idx]
        # marcar como 0 todas las filas con time < t0+1
        mask = (times >= t0) & (times < t0 + 1)
        df_mod.loc[mask, "Beam"] = 0

    return df_mod

---
## Función `import_UM(file_path)`

**Finalidad:**  
Esta función importa un archivo `.txt` que contiene los valores de **Unidades Monitor (UM)** de un paciente, generado previamente por la función `import_queries`.  
Cada línea del archivo corresponde a un campo de irradiación distinto.  
A partir de estos valores, la función calcula el tiempo mínimo de irradiación por campo (`t_camp = UM / 10`) y devuelve un `DataFrame` con el formato estándar utilizado en el procesamiento de sesiones DIBH.


**Entradas:**  
- `file_path` → Ruta completa del archivo `.txt` que contiene las Unidades Monitor del paciente.  
  Ejemplo:  r"C:\Users\Mario\Desktop\Fisica\TFG Clinic\Mario\DIBH\Queries\4806222\4806222_UM.txt"
  

**Formato del archivo esperado:**  
El archivo debe contener **una lista de valores de UM**, uno por línea, sin encabezados ni separadores adicionales.  
Ejemplo de contenido:

150
200
180

Cada número representa las Unidades Monitor (UM) administradas en un campo del tratamiento.


**Variables internas:**  
- `um_values` → Lista de valores numéricos leídos desde el archivo (convertidos a tipo `float`).  
- `t_camp` → Lista con el tiempo mínimo de irradiación de cada campo, calculado como `UM / 10`.  
- `df_UM` → DataFrame resultante con dos columnas:  
  | Columna | Descripción |  
  |----------|--------------|  
  | `UM` | Unidades Monitor de cada campo |  
  | `t_camp` | Tiempo mínimo de irradiación (UM / 10) |


**Lógica del código:**  
1. Se abre el archivo `.txt` y se leen todas las líneas con valores numéricos.  
2. Se eliminan posibles líneas vacías y se convierten los valores a `float`.  
3. Se calcula el tiempo mínimo (`t_camp`) dividiendo cada valor de `UM` entre 10 (dado que la tasa de dosis es de **10 UM/s**).  
4. Se construye un `DataFrame` con las columnas `'UM'` y `'t_camp'`.  
5. Se devuelve el `DataFrame` para ser utilizado en la segmentación de campos (`detect_groups`).


**Salida:**  
- `df_UM` → `DataFrame` con las columnas:  
  | UM | t_camp |  
  |----|---------|  
  | 150.0 | 15.0 |  
  | 200.0 | 20.0 |  
  | 180.0 | 18.0 |  


**Uso posterior:**
El DataFrame obtenido (df_UM) se emplea directamente en la función detect_groups(df_data, df_UM) para segmentar los datos de la sesión según los tiempos mínimos de irradiación definidos por las Unidades Monitor de cada campo

In [ ]:
def import_UM(file_path):

    # Leer las UM (una por línea)
    with open(file_path, 'r') as f:
        um_values = [float(line.strip()) for line in f if line.strip()]

    # Crear DataFrame
    df_UM = pd.DataFrame({
        'UM': um_values,
        't_camp': [um / 10 for um in um_values]
    })

    return df_UM

---
## Función `detect_groups(df_data, df_UM, lookahead_s=5.0, verbose=True)`

**Finalidad:**  
Esta función identifica y etiqueta los distintos **campos (grupos)** de un tratamiento DIBH dentro del registro temporal de la sesión, basándose en los intervalos en los que el haz (`Beam`) está activo y en los tiempos mínimos por campo (`t_camp`) calculados a partir de las Unidades Monitor (UM).  
Incluye además un **modo de depuración** (`verbose=True`) que imprime en consola los intervalos de tiempo asignados a cada grupo, permitiendo comprobar visualmente si la segmentación es coherente con la realidad del tratamiento.


**Entradas:**  
- `df_data` → `DataFrame` con las columnas:  
  `['Time', 'Amplitude', 'Beam', 'UpperThreshold', 'LowerThreshold']`  
  (Resultado del procesamiento tras aplicar `merge_timelines` y `cut_delay`).

- `df_UM` → `DataFrame` con las columnas:  
  `['UM', 't_camp']`, donde `t_camp` es el tiempo mínimo de irradiación para cada campo.

- `lookahead_s` → *(float, opcional, por defecto 5.0)*  
  Ventana temporal (en segundos) que permite incluir activaciones posteriores cercanas dentro del mismo campo.  
  Esto evita dividir un mismo campo si se produce una breve salida de los márgenes de respiración.

- `verbose` → *(bool, opcional, por defecto True)*  
  Si está activado, la función imprime por consola un resumen de cada grupo detectado:
  - Número de grupo  
  - Tiempo de inicio (`start_time`) y fin (`end_time`)  
  - Duración total del campo  
  - Si se aplicó el criterio de *lookahead* o si el campo fue incompleto por fin de datos  


**Lógica del algoritmo:**  
1. **Identificación de intervalos activos:**  
   Se buscan los tramos consecutivos donde `Beam == 1` (inicio y fin de cada activación).  

2. **Acumulación por campo:**  
   Se van sumando las duraciones de cada activación hasta alcanzar el tiempo mínimo de irradiación (`t_camp`) del campo actual.  
3. **Agrupación inteligente:**  
   Si tras alcanzar `t_camp` se detecta otra activación dentro de los siguientes `lookahead_s` segundos, se considera que **pertenece al mismo campo** (para evitar falsos cambios).  

4. **Fin de datos:**  
   Si se está procesando el último campo y el archivo termina antes de alcanzar su `t_camp`, se asigna el campo hasta la última desactivación (`Beam` pasa de 1 a 0).  

5. **Asignación de grupos:**  
   A cada muestra se le asigna el número de grupo correspondiente (1, 2, 3, …).  
   Los puntos no asignados quedan con valor `0` en la columna `Grupo`.  

6. **Mensajes de depuración (`verbose=True`):**  
   Por cada campo detectado, se muestra una línea en consola como esta:
   🟢 Grupo 1: 12.00s → 45.20s (duración = 33.20s, lookahead aplicado: True)
   🟡 Último grupo (3): datos terminados antes de t_camp. Asigna hasta 123.00s
   
---

   **Salidas:**  
    - `DataFrame` con las columnas:  
    | Columna | Descripción |  
    |----------|--------------|  
    | `Time` | Tiempo de adquisición de cada muestra |  
    | `Amplitude` | Señal respiratoria medida |  
    | `Beam` | Estado del haz (1 = activo, 0 = inactivo) |  
    | `Grupo` | Campo asignado (1, 2, 3, …) o 0 si no pertenece a ningún campo |  
    | `UpperThreshold` | Límite superior del rango de inspiración |  
    | `LowerThreshold` | Límite inferior del rango de inspiración |  

---



In [ ]:
def detect_groups(df_data, df_UM, lookahead_s=3.0, verbose=True):

    df = df_data.copy().reset_index(drop=True)
    df['Grupo'] = 0

    if 'Time' not in df.columns or 'Beam' not in df.columns:
        raise ValueError("df_data debe contener las columnas 'Time' y 'Beam'")

    times = df['Time'].to_numpy(dtype=float)
    beam = df['Beam'].to_numpy(dtype=int)
    n_rows = len(df)

    # --- Detectar intervalos donde beam == 1 (start_idx, end_idx) inclusive ---
    intervals = []
    i = 0
    while i < n_rows:
        if beam[i] == 1:
            start = i
            j = i
            while j + 1 < n_rows and beam[j + 1] == 1:
                j += 1
            end = j
            intervals.append((start, end))
            i = j + 1
        else:
            i += 1

    if len(intervals) == 0 or len(df_UM) == 0:
        return df[['Time', 'Amplitude', 'Beam', 'Grupo', 'UpperThreshold', 'LowerThreshold']]

    n_groups = len(df_UM)
    group_idx = 0
    interval_ptr = 0

    while interval_ptr < len(intervals) and group_idx < n_groups:
        group_start_idx = intervals[interval_ptr][0]
        accumulated_time = 0.0
        last_included_end = None

        while interval_ptr < len(intervals) and accumulated_time < float(df_UM.iloc[group_idx]['t_camp']):
            s, e = intervals[interval_ptr]
            duration = float(times[e]) - float(times[s])
            accumulated_time += duration
            last_included_end = e
            interval_ptr += 1

        # Caso 1: alcanzado t_camp
        if accumulated_time >= float(df_UM.iloc[group_idx]['t_camp']):
            # Buscar activaciones cercanas dentro de ventana lookahead_s
            while interval_ptr < len(intervals):
                next_start = intervals[interval_ptr][0]
                gap = float(times[next_start]) - float(times[last_included_end])
                if gap <= lookahead_s:
                    last_included_end = intervals[interval_ptr][1]
                    interval_ptr += 1
                else:
                    break

            df.loc[group_start_idx:last_included_end, 'Grupo'] = group_idx + 1

            if verbose:
                start_t = float(times[group_start_idx])
                end_t = float(times[last_included_end])
                print(f"🟢 Grupo {group_idx + 1}: {start_t:.2f}s → {end_t:.2f}s "
                      f"(duración = {end_t - start_t:.2f}s, lookahead aplicado: {gap <= lookahead_s})")

            group_idx += 1

        # Caso 2: últimos datos sin alcanzar t_camp
        else:
            if group_idx == n_groups - 1 and last_included_end is not None:
                df.loc[group_start_idx:last_included_end, 'Grupo'] = group_idx + 1
                if verbose:
                    start_t = float(times[group_start_idx])
                    end_t = float(times[last_included_end])
                    print(f"🟡 Último grupo ({group_idx + 1}): datos terminados antes de t_camp. "
                          f"Asigna hasta {end_t:.2f}s (duración = {end_t - start_t:.2f}s)")
            else:
                if verbose:
                    print(f"⚠️ Campo {group_idx + 1}: datos insuficientes para completar el tiempo mínimo.")
            break

    df['Grupo'] = df['Grupo'].fillna(0).astype(int)
    return df[['Time', 'Amplitude', 'Beam', 'Grupo', 'UpperThreshold', 'LowerThreshold']]

---

---
## Función `import_queries(file_path, output_dir="UM_by_ID")`

**Finalidad:**  
Esta función importa un archivo `.out` (separado por comas) que contiene información de varios pacientes y sus parámetros de tratamiento, extrae los identificadores de paciente (`ID`) y las Unidades Monitor (`UM`), y organiza los datos creando una carpeta por paciente.  
En cada carpeta se guarda un archivo `.txt` con los valores de `UM` correspondientes a ese paciente, para su posterior uso en el cálculo de los tiempos mínimos de irradiación con la función `import_UM`.


**Entradas:**  
- `file_path` → Ruta al archivo `.out` (o `.csv`) con los datos de todos los pacientes.  
  El archivo debe tener al menos 6 columnas separadas por comas, donde:
  - La **columna 2** contiene el **ID del paciente** (identificador único).  
  - La **columna 6** contiene las **Unidades Monitor (UM)** del tratamiento.  

- `output_dir` → *(str, opcional)*  
  Nombre o ruta del directorio donde se crearán las carpetas por paciente.  
  Por defecto: `"UM_by_ID"`.


**Variables internas:**  
- `df` → `DataFrame` que contiene solo las columnas seleccionadas del archivo (`ID` y `UM`).  
- `ids_unicos` → Lista de identificadores únicos de pacientes encontrados en el archivo.  
- `df_id` → Subconjunto del dataframe con las filas que corresponden a un mismo paciente.  
- `ums` → Lista de valores de `UM` de cada paciente, redondeados al entero más cercano.  
- `folder_path` → Ruta de la carpeta individual creada para el paciente dentro de `output_dir`.  
- `file_out` → Ruta completa del archivo `.txt` donde se guardan los valores `UM`.


**Lógica del código:**  
1. Se lee el archivo `.out` usando `pandas.read_csv()` con separador por comas.  
2. Se extraen únicamente las columnas **2** e **6** (índices `[1, 5]` en base cero).  
3. Se renombran las columnas a `ID` y `UM`.  
4. Se redondean las UM al entero más cercano (`round(0)` y `astype(int)`).  
5. Se crea un directorio principal (`output_dir`) si no existe.  
6. Para cada `ID` único:
   - Se genera una carpeta con su nombre (por ejemplo `UM_by_ID/4806222/`).  
   - Dentro de ella se guarda un archivo `.txt` con sus valores de `UM` (uno por línea).  
7. La función imprime por consola un resumen con el número total de carpetas creadas.  
8. Devuelve un diccionario con los IDs como claves y las listas de UM como valores.


**Salida:**  
- `dict` → Diccionario con estructura:  
  ```python
  {
      '4806222': [150, 200, 180],
      '4810090': [120, 160],
      ...
  }

Además, genera carpetas en el directorio `outputdir`

UM_by_ID/

├── 4806222/

│   └── 4806222_UM.txt

├── 4810090/

│   └── 4810090_UM.txt

├── ...


In [ ]:
def import_queries(file_path, output_dir):

    # Leer archivo separado por comas
    df = pd.read_csv(file_path, sep=',', header=None, engine='python')

    # Seleccionar solo columnas 2 y 6 (índices 1 y 5 en base 0)
    df = df.iloc[:, [1, 5]].copy()
    df.columns = ['ID', 'UM']

    # Redondear las UM al entero más cercano
    df['UM'] = df['UM'].round(0).astype(int)

    # Crear directorio base si no existe
    os.makedirs(output_dir, exist_ok=True)

    # Crear una carpeta por ID y guardar los UM en un txt
    ids_unicos = df['ID'].unique()
    resultado = {}

    for id_paciente in ids_unicos:
        # Filtrar las filas del paciente
        df_id = df[df['ID'] == id_paciente]
        ums = df_id['UM'].tolist()
        resultado[id_paciente] = ums

        # Crear carpeta del paciente
        folder_path = os.path.join(output_dir, str(id_paciente))
        os.makedirs(folder_path, exist_ok=True)

        # Guardar archivo .txt con los valores UM
        file_out = os.path.join(folder_path, f"{id_paciente}_UM.txt")
        with open(file_out, 'w') as f:
            f.write('\n'.join(map(str, ums)))

    print(f"✅ Datos exportados correctamente en el directorio: '{output_dir}'")
    print(f"   Se generaron {len(ids_unicos)} carpetas (una por ID de paciente).")

    return resultado

---
## Función `read_patients(path_in, path_pacientes)`

**Finalidad:**  
Organiza automáticamente los archivos `.txt` de los campos de respiración en la estructura de carpetas de los pacientes.  
Cada archivo tiene el formato `ID_sesion_campo.txt`, por ejemplo: `5773629_3_6.txt`, que corresponde al paciente `5773629`, sesión `3` y campo `6`.

**Funcionamiento:**  
La función busca en la carpeta `path_in` todos los archivos `.txt` y los distribuye dentro de la carpeta `path_pacientes` en su estructura correspondiente:

Pacientes/

├── 5773629/

│ ├── 5773629_UM.txt

│ ├── 5773629_1/

│ │ ├── 5773629_1_1.txt

│ │ ├── 5773629_1_2.txt

│ ├── 5773629_2/

│ │ ├── ...

**Inputs:**  
- `path_in`: ruta donde están los archivos `.txt` de entrada.  
- `path_pacientes`: ruta de la carpeta principal `"Pacientes"` (ya generada por `import_queries`).

**Outputs:**  
No devuelve nada, pero:
- Crea las subcarpetas de cada sesión dentro de la carpeta de cada paciente.  
- Copia o mueve (puedes elegir) los `.txt` correspondientes a su lugar.  
- Si un archivo no encaja con ningún paciente o formato válido, imprime un aviso de error indicando el nombre del archivo.


In [ ]:
def read_patients(path_in, path_pacientes, move=False):

    # Obtener lista de archivos .txt
    all_files = [f for f in os.listdir(path_in) if f.endswith('.txt')]

    if not all_files:
        print("⚠️ No se encontraron archivos .txt en la carpeta de entrada.")
        return

    # Obtener IDs de pacientes que existen en la carpeta "Pacientes"
    pacientes_existentes = [p for p in os.listdir(path_pacientes) if os.path.isdir(os.path.join(path_pacientes, p))]

    print(f"📂 Pacientes detectados: {pacientes_existentes}")
    print(f"📁 Procesando {len(all_files)} archivos...\n")

    for filename in all_files:
        match = re.match(r"(\d+)_([0-9]+)_([0-9]+)\.txt", filename)
        if not match:
            print(f"❌ Formato incorrecto en nombre de archivo: {filename}")
            continue

        paciente_id, sesion, campo = match.groups()
        paciente_path = os.path.join(path_pacientes, paciente_id)

        # Verificar si el paciente existe
        if paciente_id not in pacientes_existentes:
            print(f"⚠️ No se encontró carpeta para el paciente {paciente_id}. Archivo omitido: {filename}")
            continue

        # Crear carpeta de sesión si no existe
        sesion_folder = f"{paciente_id}_{sesion}"
        sesion_path = os.path.join(paciente_path, sesion_folder)
        os.makedirs(sesion_path, exist_ok=True)

        # Mover o copiar el archivo
        src = os.path.join(path_in, filename)
        dst = os.path.join(sesion_path, filename)

        try:
            if move:
                shutil.move(src, dst)
            else:
                shutil.copy2(src, dst)
            print(f"✅ Archivo {filename} → {sesion_folder}")
        except Exception as e:
            print(f"❌ Error moviendo {filename}: {e}")

    print("\n✅ Reorganización completada.")

---------------  
## Función `analyze_data(base_path)`

**Finalidad:**  
Automatiza el análisis completo de todas las curvas respiratorias de múltiples pacientes y sesiones.  
Recorre la carpeta principal `"Pacientes"` y, para cada paciente, procesa todas sus sesiones de tratamiento aplicando las funciones desarrolladas previamente:  
`import_UM()`, `merge_timelines()`, `cut_delay()`, `detect_groups()`.

**Flujo de trabajo:**  
1. Busca las carpetas de cada paciente dentro de `base_path`.  
2. Para cada paciente:
   - Carga el archivo `ID_UM.txt` con la función `import_UM()`.
   - Itera sobre las carpetas de sesiones (`ID_1`, `ID_2`, ...).  
   - En cada sesión:
     - Fusiona los archivos de los distintos campos (`merge_timelines()`).
     - Recorta delays iniciales y finales (`cut_delay()`).
     - Detecta grupos respiratorios válidos (`detect_groups()`).
     - Guarda el dataframe de grupos como `ID_X_groups.txt`.
  
3. Devuelve un `DataFrame` resumen con la información procesada de todas las sesiones.

**Input:**  
- `base_path` *(str)* → Ruta completa de la carpeta `"Pacientes"`.

**Output:**  
- Devuelve un `DataFrame` con las columnas:
  - `Paciente`: ID del paciente.  
  - `Sesión`: Nombre de la carpeta de sesión.  
  - `NumCampos`: Número de archivos de campos analizados.  
  - `NumGrupos`: Número de grupos respiratorios detectados.  
  - `RutaSesion`: Ruta completa de la sesión.  
  - `Resultado`: "OK" o descripción del error si falló la sesión.  

**Formato de salida generado:**  
La función crea o actualiza los siguientes archivos en cada carpeta de sesión:


In [ ]:
def analyze_data(base_path):

    resumen = []  # lista de diccionarios para almacenar resultados por sesión

    # Iterar sobre las carpetas de pacientes
    for patient_id in sorted(os.listdir(base_path)):
        patient_path = os.path.join(base_path, patient_id)
        if not os.path.isdir(patient_path):
            continue

        print(f"\n🔍 Analizando paciente: {patient_id}")

        um_file = os.path.join(patient_path, f"{patient_id}_UM.txt")
        if not os.path.exists(um_file):
            print(f"⚠️ No se encontró {um_file}, se omite este paciente.")
            continue

        try:
            df_UM = import_UM(um_file)
        except Exception as e:
            print(f"❌ Error leyendo {um_file}: {e}")
            continue

        # Iterar sobre las carpetas de sesiones (ID_1, ID_2, ...)
        for session in sorted(os.listdir(patient_path)):
            session_path = os.path.join(patient_path, session)
            if not os.path.isdir(session_path):
                continue

            print(f"  🩺 Procesando sesión: {session}")

            # Buscar archivos .txt de campos en la sesión
            field_files = sorted([
                os.path.join(session_path, f)
                for f in os.listdir(session_path)
                if f.endswith(".txt") and not f.endswith("_UM.txt")
            ])

            if not field_files:
                print(f"  ⚠️ No se encontraron archivos de campos en {session_path}")
                resumen.append({
                    "Paciente": patient_id,
                    "Sesión": session,
                    "NumCampos": 0,
                    "NumGrupos": None,
                    "RutaSesion": session_path,
                    "Resultado": "Sin campos"
                })
                continue
            
            try:
                # 1️⃣ Fusionar timelines
                df_data = merge_timelines(field_files)

                # 2️⃣ Recortar delay
                df_data = cut_delay(df_data)

                # 3️⃣ Detectar grupos respiratorios
                df_groups = detect_groups(df_data, df_UM)

                # 4️⃣ Guardar resultados
                groups_path = os.path.join(session_path, f"{session}_processed-data.txt")
                df_groups.to_csv(groups_path, index=False, sep="\t")

                print(f"  ✅ Sesión procesada correctamente.")

                resumen.append({
                    "Paciente": patient_id,
                    "Sesión": session,
                    "NumCampos": len(field_files),
                    "NumGrupos": len(df_groups),
                    "RutaSesion": session_path,
                    "Resultado": "OK"
                })

            except Exception as e:
                print(f"  ❌ Error procesando {session}: {e}")
                resumen.append({
                    "Paciente": patient_id,
                    "Sesión": session,
                    "NumCampos": len(field_files),
                    "NumGrupos": None,
                    "RutaSesion": session_path,
                    "Resultado": str(e)
                })

    # Convertir la lista de resultados en DataFrame resumen
    df_resumen = pd.DataFrame(resumen)
    print("\n📋 Procesamiento finalizado.")
    #display(df_resumen)

    return df_resumen

---
# 2 - PARAMETER EXTRACTION

---
## 🔍 Función `tratar_fechas`

**Objetivo:**  
Analizar la coherencia temporal de los archivos de campo (`.txt`) de cada paciente, verificando que los campos dentro de cada sesión y las sesiones dentro de cada paciente estén en orden cronológico correcto.


### 📁 Estructura de carpetas esperada

main_folder/
│

├── 1234/ ← Carpeta del paciente (patientID)

│ ├── 1234_1/ ← Sesión 1 del paciente

│ │ ├── 1234_1_1.txt ← Campo 1

│ │ ├── 1234_1_2.txt ← Campo 2

│ │ └── (otros archivos ignorados)

│ │

│ ├── 1234_2/ ← Sesión 2

│ │ ├── 1234_2_1.txt

│ │ ├── 1234_2_2.txt

│ │ └── ...

│ └── ...

│

├── 5678/

│ ├── 5678_1/

│ ├── 5678_2/

│ └── ...

└── ...


- Cada **paciente** tiene una carpeta con su `patientID` como nombre.  
- Dentro, cada **sesión** se nombra como `patientID_sesion` (por ejemplo, `5691706_5`).  
- Dentro de cada sesión, hay varios **campos de tratamiento** con formato `patientID_sesion_campo.txt`.  
- Cualquier otro archivo `.txt` que no cumpla ese formato será **ignorado** automáticamente.


### ⚙️ Funcionamiento

1. **Recorre todos los pacientes** dentro del directorio principal (`main_folder`).
2. Para cada paciente:
   - Analiza todas sus **sesiones**, ordenadas correctamente por número (1, 2, 3, …, 10, 11, …).
   - Dentro de cada sesión:
     - Procesa únicamente los archivos con formato válido `patientID_sesion_campo.txt`.
     - Lee la **línea 12** del archivo (`Started: dd/mm/yyyy. hh:mm:ss`).
     - Muestra la línea leída por pantalla.
     - Verifica que los campos están **en orden cronológico** según la hora de inicio.
   - Al final de cada sesión, muestra si los campos están ordenados correctamente.
   - Al final de cada paciente, muestra si las sesiones están **en orden cronológico creciente** (sin exigir que sean días consecutivos).
3. Finalmente, imprime un **resumen general** con:
   - Fecha más antigua y más reciente encontradas.
   - Número total de pacientes analizados.
   - Número total de sesiones analizadas.


### 🧾 Ejemplo de salida por consola

🩺 Analizando paciente 5691706...

➜ Sesión 5:
5691706_5_1.txt: Started: 25/01/2025. 09:12:03
5691706_5_2.txt: Started: 25/01/2025. 09:19:48
✅ Campos en orden cronológico correcto.

➜ Sesión 6:
5691706_6_1.txt: Started: 26/01/2025. 09:11:59
✅ Campos en orden cronológico correcto.
✅ Sesiones en orden cronológico correcto.

📅 Todas las sesiones del paciente 5691706 están en orden cronológico correcto.

📊 Resumen general:

Fecha más antigua: 25/01/2025 09:12:03

Fecha más reciente: 03/02/2025 10:08:55

Total de pacientes: 10

Total de sesiones: 145


### 🧠 Notas

- No devuelve ningún valor, solo muestra **mensajes informativos** por consola.  
- Los errores de lectura o archivos sin la línea `Started:` se notifican con `⚠️`.  
- Es ideal para **revisión manual** de la estructura temporal de los datos antes de procesarlos.


In [ ]:
import os
from datetime import datetime

def tratar_fechas(main_folder):
    formato_fecha = "%d/%m/%Y. %H:%M:%S"
    fechas_globales = []
    total_pacientes = 0
    total_sesiones = 0

    for patient_id in sorted(os.listdir(main_folder)):
        patient_path = os.path.join(main_folder, patient_id)
        if not os.path.isdir(patient_path):
            continue

        total_pacientes += 1
        print(f"\n🩺 Analizando paciente {patient_id}...")

        # 🔧 Orden correcto de las sesiones por número
        sesiones = sorted(
            [
                s for s in os.listdir(patient_path)
                if os.path.isdir(os.path.join(patient_path, s)) and s.startswith(patient_id + "_")
            ],
            key=lambda x: int(x.split('_')[-1]) if x.split('_')[-1].isdigit() else 0
        )

        fechas_sesiones = []

        for sesion in sesiones:
            sesion_path = os.path.join(patient_path, sesion)
            print(f"\n  ➜ Sesión {sesion.split('_')[-1]}:")

            # ✅ Solo archivos válidos tipo patientID_sesion_campo.txt
            campos = sorted([
                f for f in os.listdir(sesion_path)
                if f.endswith(".txt")
                and f.startswith(sesion + "_")
                and len(f.split('_')) == 3  # exactamente tres partes: ID, sesión, campo
                and f.split('_')[-1].replace('.txt', '').isdigit()
            ])

            fechas_campos = []
            for campo in campos:
                campo_path = os.path.join(sesion_path, campo)
                try:
                    with open(campo_path, "r", encoding="utf-8", errors="ignore") as file:
                        lineas = file.readlines()
                        if len(lineas) >= 12:
                            linea_fecha = lineas[11].strip()  # línea 12
                            if "Started:" in linea_fecha:
                                fecha_str = linea_fecha.split("Started:")[1].strip()
                                fecha = datetime.strptime(fecha_str, formato_fecha)
                                fechas_campos.append(fecha)
                                print(f"    {campo}: {linea_fecha}")  # 🕒 Mostrar línea 12
                            else:
                                print(f"    ⚠️ {campo}: no se encontró línea 'Started:'")
                except Exception as e:
                    print(f"    ⚠️ Error leyendo {campo}: {e}")

            # Comprobar orden interno de campos
            if len(fechas_campos) > 1:
                for i in range(1, len(fechas_campos)):
                    if fechas_campos[i] < fechas_campos[i - 1]:
                        print(f"    ❌ Orden incorrecto en campos ({campos[i-1]} → {campos[i]})")
                        break
                else:
                    print("    ✅ Campos en orden cronológico correcto.")
            elif fechas_campos:
                print("    ✅ Solo un campo, no se requiere comprobación de orden.")
            else:
                print("    ⚠️ No se encontraron archivos válidos de campo.")

            if fechas_campos:
                fecha_sesion = fechas_campos[0].date()
                fechas_sesiones.append(fecha_sesion)
                fechas_globales.extend(fechas_campos)

        # ✅ Comprobar orden cronológico entre sesiones (no consecutividad)
        if len(fechas_sesiones) > 1:
            for i in range(1, len(fechas_sesiones)):
                if fechas_sesiones[i] < fechas_sesiones[i - 1]:
                    print(f"  ❌ Sesiones desordenadas: {fechas_sesiones[i-1]} → {fechas_sesiones[i]}")
                    break
            else:
                print("  ✅ Sesiones en orden cronológico correcto.")
        elif fechas_sesiones:
            print("  ✅ Solo una sesión, no se requiere comprobación de orden.")

        # 🧾 Nuevo resumen por paciente
        if len(fechas_sesiones) > 1:
            if all(fechas_sesiones[i] >= fechas_sesiones[i - 1] for i in range(1, len(fechas_sesiones))):
                print(f"\n  📅 Todas las sesiones del paciente {patient_id} están en orden cronológico correcto.")
            else:
                print(f"\n  ⚠️ Algunas sesiones del paciente {patient_id} no están en orden cronológico.")
        else:
            print(f"\n  📅 Paciente {patient_id} tiene una sola sesión.")

        total_sesiones += len(fechas_sesiones)

    # Resultados globales
    if fechas_globales:
        fecha_mas_antigua = min(fechas_globales)
        fecha_mas_reciente = max(fechas_globales)
        print("\n📊 Resumen general:")
        print(f"   - Fecha más antigua: {fecha_mas_antigua.strftime('%d/%m/%Y %H:%M:%S')}")
        print(f"   - Fecha más reciente: {fecha_mas_reciente.strftime('%d/%m/%Y %H:%M:%S')}")
        print(f"   - Total de pacientes: {total_pacientes}")
        print(f"   - Total de sesiones: {total_sesiones}")
    else:
        print("\n⚠️ No se encontraron fechas válidas en ningún archivo.")


---
## 📈 Función `build_plot`

**Objetivo:**  
Generar y guardar una representación gráfica de la curva DIBH a partir de un archivo de datos procesado (`*_processed-data.txt`), mostrando la amplitud del movimiento respiratorio, las activaciones del haz de radiación (Beam) y los distintos grupos de tratamiento (Grupo), junto con los valores umbral (`UpperThreshold` y `LowerThreshold`).


### 📄 Entrada esperada

Archivo de texto (`.txt`) tabulado con formato:

| Time | Amplitude | Beam | Grupo | UpperThreshold | LowerThreshold |
|------|------------|------|--------|----------------|----------------|
| *meta-row* | ... | ... | ... | **valor** | **valor** |
| 0.0 | 0.12 | 0 | 0 | ... | ... |
| 0.1 | 0.15 | 0 | 1 | ... | ... |
| ... | ... | ... | ... | ... | ... |

- La **primera fila** contiene los nombres de las columnas (header).  
- La **segunda fila (meta-row)** contiene los valores de `UpperThreshold` y `LowerThreshold`.  
- El resto de filas representan los datos temporales medidos durante el tratamiento.


### ⚙️ Funcionamiento

1. **Lectura del archivo:**
   - Se lee el `.txt` mediante `pandas.read_csv` con separador `\t`.
   - Se extraen los valores de los umbrales (`UpperThreshold`, `LowerThreshold`) desde la **meta-row**.
   - Se eliminan las filas de cabecera y meta para quedarse solo con los datos reales.

2. **Procesamiento de datos:**
   - Se convierten las columnas numéricas (`Time`, `Amplitude`, `Beam`, `Grupo`) a tipo float.
   - Se eliminan filas inválidas (por ejemplo, donde `Time` o `Amplitude` no sean numéricos).
   - Se asegura que, si alguna columna no existe, se rellena con `NaN` para mantener la estructura del DataFrame.

3. **Generación del gráfico:**
   - Se representa:
     - `Amplitude` en azul frente al tiempo (`Time`).
     - `Beam` en rojo si contiene valores válidos.
     - Áreas coloreadas bajo la curva para cada grupo (`Grupo ≠ 0`).
   - Se añaden líneas horizontales para los **umbrales superior e inferior** (`UpperThreshold`, `LowerThreshold`).

4. **Estética y guardado:**
   - Se configuran títulos, etiquetas, cuadrícula y leyenda.
   - Se guarda el gráfico como imagen `.png` con el mismo nombre del archivo de entrada, reemplazando:
     ```
     *_processed-data.txt  →  *_plot.png
     ```
   - El gráfico no se muestra en pantalla (modo no interactivo), pero puede visualizarse activando manualmente `plt.show()`.


### 🧾 Ejemplo de salida
✅ Gráfico guardado en: 5691706_5_3_plot.png

El gráfico muestra:

- **Curva azul:** amplitud del movimiento torácico durante la respiración.
- **Curva roja:** activaciones del haz de radiación (Beam).
- **Áreas sombreadas:** intervalos de tratamiento por grupo.
- **Líneas horizontales:** umbrales superior e inferior.


### 🧠 Notas

- La función **no devuelve ningún valor**, únicamente genera y guarda la figura.  
- Si se desea mostrarla directamente, puede activarse `plt.show()` al final.  
- Es compatible con los archivos de salida generados por las funciones de procesamiento previas (`merge_timelines`, `extract_parameters`, etc.).  
- Requiere las librerías `pandas`, `numpy`, `matplotlib` e `itertools`.



In [ ]:
def build_plot(path):
    plt.ioff()  # Desactiva el modo interactivo
    
    # Leer archivo: la primera fila es header; la siguiente fila es la meta-row con thresholds
    df_raw = pd.read_csv(path, sep='\t', header=0, comment='#', skipinitialspace=True)
    
    # Extraer thresholds desde la meta-row (primera fila de datos)
    # Asumimos que siempre existe la meta-row y que tiene valores en UpperThreshold y LowerThreshold
    upper = df_raw.loc[0, 'UpperThreshold']
    lower = df_raw.loc[0, 'LowerThreshold']
    
    # Eliminar la meta-row para quedarse solo con los datos temporales reales
    df = df_raw.drop(index=0).reset_index(drop=True)
    
    # Convertir columnas a numérico (Time siempre existe según tus datos)
    df['Time'] = pd.to_numeric(df['Time'], errors='coerce')
    for col in ['Amplitude', 'Beam', 'Grupo']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        else:
            df[col] = np.nan

    # Eliminar filas donde Time o Amplitude no sean válidos
    df = df.dropna(subset=['Time', 'Amplitude']).reset_index(drop=True)
    
    # Preparar figura
    fig, ax = plt.subplots(figsize=(30,12))
    
    # Amplitud vs Time (azul)
    ax.plot(df['Time'], df['Amplitude'], label='Amplitud', color='blue', linewidth=1.5, zorder=3)
    
    # Beam vs Time (rojo) si existe/contiene valores
    if df['Beam'].notna().any():
        ax.plot(df['Time'], df['Beam'], label='Beam', color='red', linewidth=1.2, zorder=2)
    else:
        # no dibujamos beam si no hay datos
        pass
    
    # Rellenar el area bajo la curva de Amplitud por cada Grupo distinto de 0
    if 'Grupo' in df.columns and df['Grupo'].notna().any():
        unique_groups = np.unique(df['Grupo'].dropna())
        unique_groups = unique_groups[unique_groups != 0]  # eliminar ceros
        if len(unique_groups) > 0:
            color_cycle = itertools.cycle(['#87CEFA', '#FFA07A', '#98FB98', '#DDA0DD', '#FFD700', '#A9A9A9'])
            for g in unique_groups:
                mask = df['Grupo'] == g
                color = next(color_cycle)
                ax.fill_between(df['Time'], df['Amplitude'], where=mask, color=color, alpha=0.35,
                                label=f'Grupo {int(g)}', step=None, zorder=1)
    
    # Dibujar thresholds como líneas horizontales constantes
    # (convertimos a float por si vinieran como numpy types)
    try:
        upper_val = float(upper)
        ax.axhline(y=upper_val, color='purple', linestyle='--', linewidth=1, label=f'Upper Threshold ({upper_val:.3f})', zorder=4)
    except Exception:
        upper_val = None
    
    try:
        lower_val = float(lower)
        ax.axhline(y=lower_val, color='orange', linestyle='--', linewidth=1, label=f'Lower Threshold ({lower_val:.3f})', zorder=4)
    except Exception:
        lower_val = None
    
    # Estética
    ax.set_title('Curva DIBH')
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Amplitude (cm) / Beam')
    ax.grid(True, linestyle='--', alpha=0.4)
    ax.legend(loc='best')
    
    # Guardar y mostrar
    output_path = path.replace('_processed-data.txt', '_plot.png')
    plt.tight_layout()
    
    #si quieres guardar la figura en el path, esto
    plt.savefig(output_path, dpi=300)
    plt.close('all')
    
    #si quieres imprimir la figura aqui, esto:
    #plt.show()
    
    #print(f"✅ Gráfico guardado en: {output_path}")

-----------------------------------------------------------------------------------------------
# EJECUTABLES

---

### Flujo Ideal del codigo completo:

---
-Llamar función **IMPORT_QUERIES(file_path,output_dir)**:

	Esta función lee un "queries.out" y crea en "output_dir" carpetas para cada 
	"patientID" con un documento "patientID_UM.txt" en el que estan las UM de 
	cada campo.


In [ ]:
path_queries = r"C:\Users\Mario\Desktop\Fisica\TFG Clinic\TFG\DIBH\queries_DIBH.out"
output_dir = r"C:\Users\Mario\Desktop\Fisica\TFG Clinic\TFG\DIBH\Pacientes_NO_entreno"
res = import_queries(path_queries, output_dir) #NO OUTPUT

---
-Llamar función **READ_PATIENTS(path_in,path_pacientes)**:

	Esta función analiza una carpeta "path_in" en la que estan los txt de
	las M sesiónes de los N pacientes, genera M carpetas en el "path_pacientes"
	dentro de las N carpetas creadas anteriormente con IMPORT_QUERIES y ordena 
	todos los txt en sus correspondientes carpetas.

In [ ]:
path_in=r"C:\Users\Mario\Desktop\Fisica\TFG Clinic\TFG\DIBH\ESTUDI DIBH"
path_pacientes=r"C:\Users\Mario\Desktop\Fisica\TFG Clinic\TFG\DIBH\Pacientes_NO_entreno"
read_patients(path_in,path_pacientes)

---
-Llamar función **TREAT_DATES(base_path)**:

	Esta función recibe un "base_path", que es el directorio donde estan las N 
	carpetas de pacientes. La idea de esta función es que repase las fechas de 
	los archivos camp para comprobar que cada camp esta en su carpeta sesion
	correcta. Tambien repasa cada carpeta sesion para comprobar que los archivos
	camp estan correctamente ordenados (segun la hora de inicio de cada camp,
	especificada en el archivo al lado de la fecha).
	Unicamente hace print de OK o no OK si esta cada archivo donde debe o no.

In [ ]:
main_folder = r"C:\Users\Mario\Desktop\Fisica\TFG Clinic\TFG\DIBH\Pacientes_NO_entreno"
tratar_fechas(main_folder)

---
---
Hasta aqui tenemos N carpetas de pacientes, dentro de cada una hay M sesiones y eldocumento de las UM, y dentro de cada carpeta de sesion estan los campos de treatment.

---

---
-Llamar función **ANALYZE_DATA(base_path)**:

	Esta función recibe el mismo "base_path" que treat_dates. El objetivo de esta 
	función es analizar cada carpeta paciente, cada subcarpeta sesión y hacer el
	"merge_timelines" de los txt de treatment y dar de vuelta un 
	"patientID_sesion_processed-data.txt" que corresponde al dataframe que da la 
	función merge_timelines.

In [ ]:
analyze_data(r"C:\Users\Mario\Desktop\Fisica\TFG Clinic\Mario\Documentació inicial\DIBH_superposat\Pacientes_NO_entreno")